In [ ]:
from openai import OpenAI
import time
import pandas as pd
from google.colab import files


# Configuration du client OpenAI
# Remplacez par votre clé API
api_key = "YOUR API KEY"
client = OpenAI(api_key=api_key)

# Prompt pour générer des molécules
prompt = """Give me 100 novel molecules that do not necessarily exist in the literature (in SMILES format) that have the following property:
* logP ≤ 3 The length of the molecule should be ≤ 100 characters. I don't want a python script or any explanation, just the molecules right away."""

# Fonction pour extraire les molécules SMILES du texte
def extract_smiles(text):
    lines = text.strip().split('\n')
    smiles_list = []
    for line in lines:
        # Nettoyer les lignes et extraire le format SMILES
        # Vérifie si la ligne contient des caractères typiques de SMILES
        if any(c in line for c in "CON()[]=#") and not line.startswith("*"):
            # Extraire juste le SMILES, en supprimant les numéros ou explications
            parts = line.split()
            for part in parts:
                if any(c in part for c in "CON()[]=#"):
                    smiles_list.append(part.strip())
                    break
    return smiles_list

# Liste pour stocker toutes les molécules
all_molecules = []

# Nombre cible de molécules
TARGET_MOLECULES = 10000

# Compteur d'appels API
api_calls = 0

print("Début de la génération des molécules...")

# Continuer jusqu'à ce qu'on ait suffisamment de molécules
while len(all_molecules) < TARGET_MOLECULES:
    try:
        api_calls += 1
        print(f"Appel API #{api_calls}... ({len(all_molecules)}/{TARGET_MOLECULES} molécules obtenues)")

        # Appel à l'API GPT-4 Turbo avec la nouvelle syntaxe
        response = client.chat.completions.create(
            model="gpt-4-turbo-preview",  # Nom du modèle GPT-4 Turbo
            messages=[
                {"role": "system", "content": "You are a helpful assistant specialized in chemistry."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=2000
        )

        # Extraire le texte de la réponse (nouvelle syntaxe)
        result_text = response.choices[0].message.content

        # Extraire les molécules SMILES
        molecules = extract_smiles(result_text)

        # Ajouter à la liste principale
        all_molecules.extend(molecules)

        print(f"Obtenu {len(molecules)} molécules dans cet appel. Total actuel: {len(all_molecules)}")

        # Pause pour respecter les limites de l'API
        time.sleep(3)

    except Exception as e:
        print(f"Erreur lors de l'appel #{api_calls}: {str(e)}")
        time.sleep(20)  # Attente plus longue en cas d'erreur

# Limiter à exactement 10 000 molécules si on en a plus
final_molecules = all_molecules[:TARGET_MOLECULES]

# Créer un DataFrame avec les molécules (en gardant les doublons)
df = pd.DataFrame(final_molecules, columns=["SMILES"])

# Ajouter une colonne qui compte les occurrences pour l'analyse de performance
occurrence_counts = {}
for smiles in final_molecules:
    occurrence_counts[smiles] = occurrence_counts.get(smiles, 0) + 1

df["occurrences"] = df["SMILES"].map(occurrence_counts)

# Enregistrer dans un fichier CSV
filename = "molecules_generated.csv"
df.to_csv(filename, index=False)

# Télécharger le fichier
files.download(filename)

# Statistiques finales
unique_count = len(set(final_molecules))
duplicate_rate = (1 - unique_count/len(final_molecules)) * 100

print(f"Génération terminée.")
print(f"Nombre d'appels API effectués: {api_calls}")
print(f"Nombre total de molécules: {len(final_molecules)}")
print(f"Nombre de molécules uniques: {unique_count}")
print(f"Taux de duplication: {duplicate_rate:.2f}%")
print(f"Fichier sauvegardé: {filename}")

In [ ]:
from openai import OpenAI
import time
# pandas n'est plus nécessaire puisqu'on utilise un fichier txt
# import pandas as pd
from google.colab import files


# Configuration du client OpenAI
# Remplacez par votre clé API
api_key = "YOUR API KEY"
client = OpenAI(api_key=api_key)

# Prompt pour générer des molécules
prompt = """Give me 100 novel molecules that do not necessarily exist in the literature (in SMILES format) that have the following property:
* logP ≤ 3 The length of the molecule should be ≤ 100 characters. I don't want a python script or any explanation, just the molecules right away."""

# Fonction pour extraire les molécules SMILES du texte
def extract_smiles(text):
    lines = text.strip().split('\n')
    smiles_list = []
    for line in lines:
        # Nettoyer les lignes et extraire le format SMILES
        # Vérifie si la ligne contient des caractères typiques de SMILES
        if any(c in line for c in "CON()[]=#") and not line.startswith("*"):
            # Extraire juste le SMILES, en supprimant les numéros ou explications
            parts = line.split()
            for part in parts:
                if any(c in part for c in "CON()[]=#"):
                    smiles_list.append(part.strip())
                    break
    return smiles_list

# Liste pour stocker toutes les molécules
all_molecules = []

# Nombre cible de molécules
TARGET_MOLECULES = 100  # Modifié de 10000 à 100

# Compteur d'appels API
api_calls = 0

print("Début de la génération des molécules...")

# Continuer jusqu'à ce qu'on ait suffisamment de molécules
while len(all_molecules) < TARGET_MOLECULES:
    try:
        api_calls += 1
        print(f"Appel API #{api_calls}... ({len(all_molecules)}/{TARGET_MOLECULES} molécules obtenues)")

        # Appel à l'API GPT-4 Turbo avec la nouvelle syntaxe
        response = client.chat.completions.create(
            model="gpt-4-turbo-preview",  # Nom du modèle GPT-4 Turbo
            messages=[
                {"role": "system", "content": "You are a helpful assistant specialized in chemistry."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=2000
        )

        # Extraire le texte de la réponse (nouvelle syntaxe)
        result_text = response.choices[0].message.content

        # Extraire les molécules SMILES
        molecules = extract_smiles(result_text)

        # Ajouter à la liste principale
        all_molecules.extend(molecules)

        print(f"Obtenu {len(molecules)} molécules dans cet appel. Total actuel: {len(all_molecules)}")

        # Pause pour respecter les limites de l'API
        time.sleep(3)

    except Exception as e:
        print(f"Erreur lors de l'appel #{api_calls}: {str(e)}")
        time.sleep(20)  # Attente plus longue en cas d'erreur

# Limiter à exactement 100 molécules si on en a plus
final_molecules = all_molecules[:TARGET_MOLECULES]

# Enregistrer les molécules dans un fichier TXT (une molécule par ligne)
filename = "molecules_generated.txt"
with open(filename, 'w') as f:
    for smiles in final_molecules:
        f.write(f"{smiles}\n")

# Calculer également les occurrences pour les statistiques
occurrence_counts = {}
for smiles in final_molecules:
    occurrence_counts[smiles] = occurrence_counts.get(smiles, 0) + 1

# Télécharger le fichier
files.download(filename)

# Statistiques finales
unique_count = len(set(final_molecules))
duplicate_rate = (1 - unique_count/len(final_molecules)) * 100

print(f"Génération terminée.")
print(f"Nombre d'appels API effectués: {api_calls}")
print(f"Nombre total de molécules: {len(final_molecules)}")
print(f"Nombre de molécules uniques: {unique_count}")
print(f"Taux de duplication: {duplicate_rate:.2f}%")
print(f"Fichier sauvegardé: {filename}")

In [ ]:
from openai import OpenAI
import time
import os
from google.colab import files

# Configuration du client OpenAI
# Remplacez par votre clé API
api_key = "YOUR API KEY"
client = OpenAI(api_key=api_key)

# Prompt pour générer des molécules
prompt = """Give me 100 novel molecules that do not necessarily exist in the literature (in SMILES format) that have the following property:
* logP ≤ 3 The length of the molecule should be ≤ 100 characters. I don't want a python script or any explanation, just the molecules right away."""

# Fonction pour extraire les molécules SMILES du texte
def extract_smiles(text):
    lines = text.strip().split('\n')
    smiles_list = []
    for line in lines:
        # Nettoyer les lignes et extraire le format SMILES
        # Vérifie si la ligne contient des caractères typiques de SMILES
        if any(c in line for c in "CON()[]=#") and not line.startswith("*"):
            # Extraire juste le SMILES, en supprimant les numéros ou explications
            parts = line.split()
            for part in parts:
                if any(c in part for c in "CON()[]=#"):
                    smiles_list.append(part.strip())
                    break
    return smiles_list

# Nom du fichier
filename = "molecules_generated.txt"

# Fonction pour sauvegarder les molécules dans le fichier
def save_molecules(molecules_list, mode='a'):
    with open(filename, mode) as f:
        for smiles in molecules_list:
            f.write(f"{smiles}\n")
    print(f"Sauvegarde de {len(molecules_list)} molécules dans {filename}")

# Fonction pour lire les molécules existantes
def read_existing_molecules():
    if not os.path.exists(filename):
        return []

    with open(filename, 'r') as f:
        existing_molecules = [line.strip() for line in f if line.strip()]

    return existing_molecules

# Lire les molécules existantes
all_molecules = read_existing_molecules()
print(f"Lecture de {len(all_molecules)} molécules existantes depuis {filename}")

# Nombre cible de molécules
TARGET_MOLECULES = 10000
REMAINING_MOLECULES = TARGET_MOLECULES - len(all_molecules)

if REMAINING_MOLECULES <= 0:
    print(f"Vous avez déjà atteint ou dépassé le nombre cible de molécules ({len(all_molecules)}/{TARGET_MOLECULES})")
    # Limiter à exactement 10 000 molécules si on en a plus
    if len(all_molecules) > TARGET_MOLECULES:
        all_molecules = all_molecules[:TARGET_MOLECULES]
        # Réécrire le fichier avec seulement les 10 000 premières molécules
        save_molecules(all_molecules, mode='w')
        print(f"Fichier mis à jour pour contenir exactement {TARGET_MOLECULES} molécules")
else:
    print(f"Il reste {REMAINING_MOLECULES} molécules à générer pour atteindre l'objectif de {TARGET_MOLECULES}")

    # Compteur d'appels API
    api_calls = 0

    print("Début de la génération des molécules supplémentaires...")

    # Continuer jusqu'à ce qu'on ait suffisamment de molécules
    while len(all_molecules) < TARGET_MOLECULES:
        try:
            api_calls += 1
            print(f"Appel API #{api_calls}... ({len(all_molecules)}/{TARGET_MOLECULES} molécules obtenues)")

            # Appel à l'API GPT-4 Turbo avec la nouvelle syntaxe
            response = client.chat.completions.create(
                model="gpt-4-turbo-preview",  # Nom du modèle GPT-4 Turbo
                messages=[
                    {"role": "system", "content": "You are a helpful assistant specialized in chemistry."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2000
            )

            # Extraire le texte de la réponse (nouvelle syntaxe)
            result_text = response.choices[0].message.content

            # Extraire les molécules SMILES
            molecules = extract_smiles(result_text)

            # Ajouter à la liste principale
            all_molecules.extend(molecules)

            # Sauvegarder les nouvelles molécules de cet appel
            save_molecules(molecules)

            print(f"Obtenu {len(molecules)} molécules dans cet appel. Total actuel: {len(all_molecules)}")

            # Pause pour respecter les limites de l'API
            time.sleep(3)

        except Exception as e:
            print(f"Erreur lors de l'appel #{api_calls}: {str(e)}")
            time.sleep(20)  # Attente plus longue en cas d'erreur

    # Limiter à exactement 10 000 molécules si on en a plus
    if len(all_molecules) > TARGET_MOLECULES:
        extra_count = len(all_molecules) - TARGET_MOLECULES
        print(f"Objectif dépassé de {extra_count} molécules. Limitation à {TARGET_MOLECULES}.")

# S'assurer qu'on a exactement le nombre cible
final_molecules = all_molecules[:TARGET_MOLECULES]

# Télécharger le fichier
files.download(filename)

# Calculer les statistiques
unique_count = len(set(final_molecules))
duplicate_rate = (1 - unique_count/len(final_molecules)) * 100

# Calculer les occurrences pour les statistiques
occurrence_counts = {}
for smiles in final_molecules:
    occurrence_counts[smiles] = occurrence_counts.get(smiles, 0) + 1

print(f"Génération terminée.")
print(f"Nombre total de molécules: {len(final_molecules)}")
print(f"Nombre de molécules uniques: {unique_count}")
print(f"Taux de duplication: {duplicate_rate:.2f}%")
print(f"Fichier sauvegardé: {filename}")

In [ ]:
from openai import OpenAI
import time
import os
from google.colab import files

# Configuration du client OpenAI
# Remplacez par votre clé API
api_key = "YOUR API KEY"
client = OpenAI(api_key=api_key)

# Prompt pour générer des molécules
prompt = """Give me 100 novel molecules that do not necessarily exist in the literature (in SMILES format) with the following combined properties:
* Exactly 2 aromatic rings and exactly 1 non-aromatic ring
* At least one of the following functional groups: -OH, -COOR, -COOH, or -NH2
* R-value within [0.05–0.50]
* logP ≤ 3
* Molecular weight ≤ 480 g/mol
* Hydrogen bond acceptors ≤ 3
* Hydrogen bond donors ≤ 3
* Rotational bonds ≤ 3 The length of the molecule should be ≤ 100 characters. I don't want a python script or any explanation, just the molecules right away."""

# Fonction pour extraire les molécules SMILES du texte
def extract_smiles(text):
    lines = text.strip().split('\n')
    smiles_list = []
    for line in lines:
        # Nettoyer les lignes et extraire le format SMILES
        # Vérifie si la ligne contient des caractères typiques de SMILES
        if any(c in line for c in "CON()[]=#") and not line.startswith("*"):
            # Extraire juste le SMILES, en supprimant les numéros ou explications
            parts = line.split()
            for part in parts:
                if any(c in part for c in "CON()[]=#"):
                    smiles_list.append(part.strip())
                    break
    return smiles_list

# Nom du fichier
filename = "GeneratedMoleculeObjectif4.txt"

# Fonction pour sauvegarder les molécules dans le fichier
def save_molecules(molecules_list, mode='a'):
    with open(filename, mode) as f:
        for smiles in molecules_list:
            f.write(f"{smiles}\n")
    print(f"Sauvegarde de {len(molecules_list)} molécules dans {filename}")

# Fonction pour lire les molécules existantes
def read_existing_molecules():
    if not os.path.exists(filename):
        return []

    with open(filename, 'r') as f:
        existing_molecules = [line.strip() for line in f if line.strip()]

    return existing_molecules

# Créer un nouveau fichier vide
with open(filename, 'w') as f:
    pass
print(f"Création d'un nouveau fichier {filename}")

# Initialiser la liste de molécules vide
all_molecules = []

# Nombre cible de molécules
TARGET_MOLECULES = 10000
print(f"Objectif: générer {TARGET_MOLECULES} molécules")

# Compteur d'appels API
api_calls = 0

print("Début de la génération des molécules...")

# Continuer jusqu'à ce qu'on ait suffisamment de molécules
while len(all_molecules) < TARGET_MOLECULES:
    try:
        api_calls += 1
        print(f"Appel API #{api_calls}... ({len(all_molecules)}/{TARGET_MOLECULES} molécules obtenues)")

        # Appel à l'API GPT-4 Turbo avec la nouvelle syntaxe
        response = client.chat.completions.create(
            model="gpt-4-turbo-preview",  # Nom du modèle GPT-4 Turbo
            messages=[
                {"role": "system", "content": "You are a helpful assistant specialized in chemistry."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=2000
        )

        # Extraire le texte de la réponse (nouvelle syntaxe)
        result_text = response.choices[0].message.content

        # Extraire les molécules SMILES
        molecules = extract_smiles(result_text)

        # Ajouter à la liste principale
        all_molecules.extend(molecules)

        # Sauvegarder les nouvelles molécules de cet appel
        save_molecules(molecules)

        print(f"Obtenu {len(molecules)} molécules dans cet appel. Total actuel: {len(all_molecules)}")

        # Pause pour respecter les limites de l'API
        time.sleep(3)

    except Exception as e:
        print(f"Erreur lors de l'appel #{api_calls}: {str(e)}")
        time.sleep(20)  # Attente plus longue en cas d'erreur

# Limiter à exactement 10 000 molécules si on en a plus
if len(all_molecules) > TARGET_MOLECULES:
    extra_count = len(all_molecules) - TARGET_MOLECULES
    print(f"Objectif dépassé de {extra_count} molécules. Limitation à {TARGET_MOLECULES}.")

# S'assurer qu'on a exactement le nombre cible
final_molecules = all_molecules[:TARGET_MOLECULES]

# Télécharger le fichier
files.download(filename)

# Calculer les statistiques
unique_count = len(set(final_molecules))
duplicate_rate = (1 - unique_count/len(final_molecules)) * 100

# Calculer les occurrences pour les statistiques
occurrence_counts = {}
for smiles in final_molecules:
    occurrence_counts[smiles] = occurrence_counts.get(smiles, 0) + 1

print(f"Génération terminée.")
print(f"Nombre total de molécules: {len(final_molecules)}")
print(f"Nombre de molécules uniques: {unique_count}")
print(f"Taux de duplication: {duplicate_rate:.2f}%")
print(f"Fichier sauvegardé: {filename}")